In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
imdb = keras.datasets.imdb

 # num_words=10000的意思是训练集中我们指保留词频最高的前10000个单词。
 # 10000名之后的词汇都会被直接忽略，不出现在train_data和test_data中。
vocab_size = 10000
index_from = 3 

# 词的索引+3，因为数据集中词的索引从1开始，向右挪3个位置，
# 前面从0算起留出4个槽位，可以放0:【PAD】,1:【STAR】,2:【UNK】,3:【END】
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words = vocab_size, index_from = index_from)

In [ ]:
print(train_data[0], train_labels[0])
print(train_data[1], train_labels[1])
print(train_data.shape, train_labels.shape)
print(len(train_data[0]), len(train_data[1]))
# [1, 14, 22, 16, 43,...]词的索引,这是除了开头1标志位，其他位置词语的索引都往右偏移3个位置后的结果
# 1表示起始没有实际意义，14是词的索引是原始索引11+index_from:3=14

In [ ]:
print(test_data.shape, test_labels.shape)

In [ ]:
# 加载词典 
word_index = imdb.get_word_index()
print(len(word_index))
print(word_index)
# 词对应的索引从1开始：the:1,and:2...

In [ ]:
for k,v in word_index.items():
    if k=="fawn":
        print(v)

In [ ]:
# 把词的索引向右挪三个位置
word_index = {k:(v+3) for k, v in word_index.items()}

In [ ]:
# 前面空出的槽位，添加4个标记
word_index['<PAD>'] = 0 # padding 填充位
word_index['<START>'] = 1 # 起始位
word_index['<UNK>'] = 2 # 空值
word_index['<END>'] = 3 # 结束标志

# 将词典中key、value置换
reverse_word_index = dict(
    [(value, key) for key, value in word_index.items()])


In [ ]:

def decode_review(text_ids):
    """解析训练数据的内容"""
    return ' '.join(
        [reverse_word_index.get(word_id, "<UNK>") for word_id in text_ids])

In [ ]:
print(train_data[0])

In [ ]:
decode_review(train_data[0])

In [ ]:
print(reverse_word_index[14],reverse_word_index[22])

In [ ]:
for k,v in word_index.items():
    if v==0:
        print(k)

In [ ]:
max_length = 500 # 句子高于500被截断，低于500被补全

# 进行数据补全
train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, # list of list
    value = word_index['<PAD>'],# 用PAD的索引值0填充
    padding = 'post', # 不够max_length，post:往句子后填充, pre:向句子前方进行填充 
    maxlen = max_length)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data, # list of list
    value = word_index['<PAD>'],
    padding = 'post', # post, pre
    maxlen = max_length)

print(train_data[0])

In [ ]:
# 定义模型
embedding_dim = 16 # 每个词embedding（转换）成长度为16的向量
batch_size = 128

model = keras.models.Sequential([
    # 1. define matrix: [vocab_size, embedding_dim]
    # 2. [1,2,3,4..], max_length * embedding_dim
    # 3. batch_size * max_length * embedding_dim
    keras.layers.Embedding(vocab_size, embedding_dim,
                           input_length = max_length),
    # batch_size * max_length * embedding_dim 
    #   -> batch_size * embedding_dim
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid'),
])

model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_data, train_labels,
                    epochs = 30,
                    batch_size = batch_size,
                    validation_split = 0.2)

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'accuracy', 30, 0, 1)
plot_learning_curves(history, 'loss', 30, 0, 1)

- 验证集的loss到达一定次数后逐步上升
- 训练集的loss一直在稳步下降
- 说明随着训练次数增加出现过拟合的现象

In [ ]:
model.evaluate(
    test_data, test_labels,
    batch_size = batch_size,)